## Joining on Non-Equi Operators

In [1]:
import pandas as pd
import janitor
import numpy as np

In [17]:
from io import StringIO

data = """ A    B  C      D          E
    0  0   0.0  a  False  1970-01-01
    """

df = pd.read_csv(StringIO(data),sep='\s{2,}', parse_dates=['E'], engine='python')
df

,A,B,C,D,E
0,0,-1.0,a,False,1970-01-01
1,0,0.0,a,False,1970-01-01
2,0,-1.0,a,False,1970-01-01


In [22]:
data = """Integers  Numeric  Floats   Strings  Booleans      Dates   Dates_Right
    0         0      0.0     0.0       a      False   1970-01-01  1970-01-01
    1        -1      0.0     0.0       a      False   1970-01-01  1970-01-01
"""

right = pd.read_csv(StringIO(data),sep='\s{2,}', parse_dates=['Dates', 'Dates_Right'], engine='python')
right

,Integers,Numeric,Floats,Strings,Booleans,Dates,Dates_Right
0,0,0.0,0.0,a,False,1970-01-01,1970-01-01
1,-1,0.0,0.0,a,False,1970-01-01,1970-01-01


In [25]:
l_eq, l_ge, l_le = ["B", "A", "E"]
r_eq, r_ge, r_le = ["Floats", "Integers", "Dates"]
columns = ["B", "A", "E", "Floats", "Integers", "Dates"]
expected = (
    df.merge(right, left_on=l_eq, right_on=r_eq, how="inner", sort=False)
    .dropna(subset=[l_eq, r_eq])
    .query(f"{l_ge} >= {r_ge} and {l_le} <= {r_le}")
    .reset_index(drop=True)
)
expected = expected.filter(columns)
expected



,B,A,E,Floats,Integers,Dates
0,0.0,0,1970-01-01,0.0,0,1970-01-01
1,0.0,0,1970-01-01,0.0,-1,1970-01-01


In [27]:
actual = df.conditional_join(
    right,
    (l_ge, r_ge, ">="),
    (l_le, r_le, "<="),
    (l_eq, r_eq, "=="),
    how="inner",
    sort_by_appearance=True,
)
# actual = actual.droplevel(0, 1)
actual = actual.filter(columns)
actual

,B,A,E,Floats,Integers,Dates
0,0.0,0,1970-01-01,0.0,0,1970-01-01
1,0.0,0,1970-01-01,0.0,-1,1970-01-01


In [6]:
#https://stackoverflow.com/q/61948103/7175713 
df1 = pd.DataFrame({'id': [1,1,1,2,2,3], 
                    'value_1': [2,5,7,1,3,4]})

df2 = pd.DataFrame({'id': [1,1,1,1,2,2,2,3], 
                    'value_2A': [0,3,7,12,0,2,3,1], 
                    'value_2B': [1,5,9,15,1,4,6,3]})

In [7]:
df1

,id,value_1
0,1,2
1,1,5
2,1,7
3,2,1
4,2,3
5,3,4


In [8]:
df2

,id,value_2A,value_2B
0,1,0,1
1,1,3,5
2,1,7,9
3,1,12,15
4,2,0,1
5,2,2,4
6,2,3,6
7,3,1,3


Join on equi and non-equi operators is possible:

In [9]:
df1.conditional_join(
        df2,
        ('id', 'id', '=='),
        ('value_1', 'value_2A', '>='),
        ('value_1', 'value_2B', '<='),
        sort_by_appearance = True
    )

left         right                  
    id value_1    id value_2A value_2B
0    1       5     1        3        5
1    1       7     1        7        9
2    2       1     2        0        1
3    2       3     2        2        4
4    2       3     2        3        6

The default join is inner. left and right joins are supported as well:

In [10]:
df1.conditional_join(
        df2,
        ('id', 'id', '=='),
        ('value_1', 'value_2A', '>='),
        ('value_1', 'value_2B', '<='),
        how='left',
        sort_by_appearance = True
    )

left         right                  
    id value_1    id value_2A value_2B
0    1       2   NaN      NaN      NaN
1    1       5   1.0      3.0      5.0
2    1       7   1.0      7.0      9.0
3    2       1   2.0      0.0      1.0
4    2       3   2.0      2.0      4.0
5    2       3   2.0      3.0      6.0
6    3       4   NaN      NaN      NaN

In [11]:
df1.conditional_join(
        df2,
        ('id', 'id', '=='),
        ('value_1', 'value_2A', '>='),
        ('value_1', 'value_2B', '<='),
        how='right',
        sort_by_appearance = True
    )

left         right                  
    id value_1    id value_2A value_2B
0  NaN     NaN     1        0        1
1  1.0     5.0     1        3        5
2  1.0     7.0     1        7        9
3  NaN     NaN     1       12       15
4  2.0     1.0     2        0        1
5  2.0     3.0     2        2        4
6  2.0     3.0     2        3        6
7  NaN     NaN     3        1        3

Join on just the non-equi joins is also possible:

In [12]:
df1.conditional_join(
        df2,
        ('value_1', 'value_2A', '>'),
        ('value_1', 'value_2B', '<'),
        how='inner',
        sort_by_appearance = True
    )

left         right                  
    id value_1    id value_2A value_2B
0    1       2     3        1        3
1    1       5     2        3        6
2    2       3     2        2        4
3    3       4     1        3        5
4    3       4     2        3        6

Join on just equality is also possible, but should be avoided, as Pandas' merge/join is more efficient (it uses Pandas' internal merge functions anyway);

In [13]:
df1.conditional_join(
    df2,
    ('id', 'id', "==")
)

ValueError: 
            conditional_join does not support equality only joins.
            Kindly use `pd.merge` instead.
            

Join on not equal -> !=

In [ ]:
df1.conditional_join(
        df2,
        ('id', 'id', "!=")
    )

left         right                  
     id value_1    id value_2A value_2B
0     1       2     2        0        1
1     1       2     2        2        4
2     1       2     2        3        6
3     1       2     3        1        3
4     1       5     2        0        1
5     1       5     2        2        4
6     1       5     2        3        6
7     1       5     3        1        3
8     1       7     2        0        1
9     1       7     2        2        4
10    1       7     2        3        6
11    1       7     3        1        3
12    2       1     3        1        3
13    2       3     3        1        3
14    2       1     1        0        1
15    2       1     1        3        5
16    2       1     1        7        9
17    2       1     1       12       15
18    2       3     1        0        1
19    2       3     1        3        5
20    2       3     1        7        9
21    2       3     1       12       15
22    3       4     1        0        1
23    3       4     1        3        5
24    3       4     1        7        9
25    3       4     1       12       15
26    3       4     2        0        1
27    3       4     2        2        4
28    3       4     2        3        6

If the columns from both dataframes have nothing in common, a single indexed column is returned:

In [ ]:
(df1.select_columns('value_1')
    .conditional_join(
        df2.select_columns('val*'),
        ('value_1', 'value_2A', '>'),
        ('value_1', 'value_2B', '<'),
    )
)

,value_1,value_2A,value_2B
0,2,1,3
1,5,3,6
2,3,2,4
3,4,3,5
4,4,3,6
